<a href="https://colab.research.google.com/github/jakewalter/intro_seismology/blob/main/final_project_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

!wget https://jakewalter.mynetgear.com/data/data.zip
!unzip data.zip


import scipy.io as sio
sta = sio.loadmat('station.mat')
# fracout = simpl['fracoutsimple']
# fracouta = simpl['fracoutsimplea']

# Load data
M6_2017 = sio.loadmat('M6_2017.mat')
#M6_2017 = np.load('M6_2017.npy', allow_pickle=True).item()

# Load pre-existing picks
ppick_exist = sio.loadmat('ppick_exist.mat')
pick = ppick_exist['ppick'][0][1][0]

# Plot waveform for each station and pick polarity
Nst = len(M6_2017['data']['samprate'][0][0][0])
twin_frompick = [-10, 20]  # in seconds: will use waveform 2 seconds before and 10 seconds after the pick.
polarity = np.zeros(Nst)

for ist in range(Nst):
    tnwin_frompick = np.round(np.array(twin_frompick) * M6_2017['data']['samprate'][0][0][0][ist]).astype(int)  # Convert time window to samples
    tn = np.round(pick[ist] * M6_2017['data']['samprate'][0][0][0][ist]) + tnwin_frompick  # Sample index for event 1

    trace = M6_2017['data']['seis'][0][0][0][ist][int(tn[0]):int(tn[1])]  # Seismic waveform 1 at each common station
    trace = trace - np.mean(trace)  # Remove mean of the trace
    timestamp = np.arange(tnwin_frompick[0], tnwin_frompick[1]) / M6_2017['data']['samprate'][0][0][0][ist]  # Time stamp for each data point for plotting
    staname = M6_2017['data']['stid'][0][0][0][ist][0]
    plt.figure()
    plt.plot(timestamp, trace, 'k', linewidth=1.5)
    plt.plot([0, 0], [0, np.max(trace) / 10], 'r', linewidth=1)
    plt.plot(twin_frompick, [0, 0], 'r-', linewidth=1)
    plt.xlabel('time (seconds)')
    plt.ylabel('amplitude')
    plt.title(f"station: {staname}")
    plt.grid(True)
    plt.show()
    # Please enter your polarity pick in the command window
    #polarity[ist] = input(f"station {ist + 1}: polarity (1: up, -1: down, 0: unclear): ")


In [ ]:
for ist in range(Nst):
    polarity[ist] = int(input(f"station {M6_2017['data']['stid'][0][0][0][ist][0]}: polarity (1: up, -1: down, 0: unclear): "))


In [ ]:

# Making polarity plot for interpretation
stlat = sta['stlat'][0]
stlon = sta['stlon'][0]
qlat = M6_2017['data']['qlat'][0][0][0][0]
qlon = M6_2017['data']['qlon'][0][0][0][0]

stdist, staz = [], []
for idx1, stid1 in enumerate(sta['stid'][0]):
    stid = stid1[0]

    dist = np.sqrt((qlat - stlat[idx1]) ** 2 + (qlon - stlon[idx1]) ** 2)
    stdist.append(dist)
    staz.append(np.rad2deg(np.arctan2(qlon - stlon[idx1], qlat - stlat[idx1])))

polar_r = np.ones(Nst)
polar_th = np.array(staz)
plt.figure()
ax = plt.subplot(111, polar=True)
for ist in range(Nst):
    if polarity[ist] == 1:
        ax.text(polar_th[ist], polar_r[ist], '+', fontsize=18)
    elif polarity[ist] == -1:
        ax.text(polar_th[ist], polar_r[ist], 'o', fontsize=18)

ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)
plt.show()